In [3]:
from src.dataset import Mimic3Dataset, padded_collate
from src.model import DST
import pdb
import numpy as np
import torch
import pytorch_lightning as pl
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

from lifelines import CoxPHFitter

#### About

Comparison of methods for computing ATE on restricted mean survival time. We used the following benchmarks:
- Outcome Regression with DynST
- Outcome Regression with Cox Model
- Logistic IPW
- AIPW (logistic propensity score model and DynST outcome model)

#### True ATE
- $\tau = 8$: 0.265
- $\tau = 12$: 0.572
- $\tau = 16$: 0.946

#### Unadjusted Treatment Effect
- $\tau = 8$: -0.237
- $\tau = 12$: -0.539
- $\tau = 16$: -0.933

### Outcome Regression

In [2]:
dataset_treated = Mimic3Dataset(".", intervention=True, seed=30)
dataset_control = Mimic3Dataset(".", intervention=False, seed=30)

In [4]:
checkpoints = ["multirun/2022-09-19/13-54-24/70/11/lightning_logs/version_0/checkpoints/epoch=3-step=3036.ckpt",
              "multirun/2022-09-19/13-54-24/71/15/lightning_logs/version_0/checkpoints/epoch=3-step=3036.ckpt",
              "multirun/2022-09-19/13-54-24/72/35/lightning_logs/version_0/checkpoints/epoch=3-step=3036.ckpt",
              "multirun/2022-09-19/13-54-37/73/1/lightning_logs/version_0/checkpoints/epoch=3-step=3036.ckpt",
              "multirun/2022-09-19/13-54-37/74/17/lightning_logs/version_0/checkpoints/epoch=2-step=2277.ckpt",
              "multirun/2022-09-19/13-54-37/75/25/lightning_logs/version_0/checkpoints/epoch=3-step=3036.ckpt"]

In [6]:
collate_fn = lambda x: padded_collate(x, pad_index=-100, causal=True)
def get_predictions(model, dataset):
    dl = torch.utils.data.DataLoader(dataset, collate_fn=collate_fn, batch_size=96)
    predictor = pl.Trainer(gpus=[5])
    predictions = predictor.predict(model, dataloaders=dl)
    return torch.cat(predictions)

In [7]:
predict_treated = []
predict_control = []
seeds = [70,71,72,73,74,75]
for ix, c in enumerate(checkpoints):
    print(seeds[ix])
    model = DST.load_from_checkpoint(c)
    predict_treated.append(get_predictions(model, dataset_treated))
    predict_control.append(get_predictions(model, dataset_control))

70


/home/pchatha/.cache/pypoetry/virtualenvs/mimic-slGwTqhJ-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[5])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[5])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/pchatha/.cache/pypoetry/virtualenvs/mimic-slGwTqhJ-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_

Predicting: 0it [00:00, ?it/s]

/home/pchatha/DynST/src/dataset.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["codes"] = torch.tensor(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

...

75


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

#### Outcome Regression Estimate

In [9]:
taus = [8, 12, 16]
for i in range(len(predict_treated)):
    print(i)
    for tau in taus:    
        cutoff = torch.full(predict_treated[0].shape, tau)
        ey_x1 = torch.minimum(cutoff, predict_treated[i])
        ey_x0 = torch.minimum(cutoff, predict_control[i])
        print(ey_x1.mean() - ey_x0.mean())

0
tensor(0.1700)
tensor(0.4110)
tensor(0.6255)
1
tensor(0.0733)
tensor(0.3657)
tensor(0.7719)
2
tensor(0.1028)
tensor(0.3217)
tensor(0.6130)
3
tensor(0.0900)
tensor(0.3784)
tensor(0.7639)
4
tensor(0.1680)
tensor(0.5457)
tensor(0.8904)
5
tensor(0.1077)
tensor(0.4523)
tensor(0.8688)


### Cox Regression

In [5]:
df = pd.read_csv("data/mimic3_df_30.csv", index_col=[0,1])
df_sub = df.drop(
    columns=["treated", "control", "hazard", "q", "survival_prob", 
             "survives", "censored","corrected_survival", "critical", "first_failure",
            "baseline_hazard"]
)
df_flat = df_sub.groupby(level=0).mean()

In [8]:
df_flat["total_hours"] = df.groupby(level=0)["corrected_survival"].sum()
df_flat["uncensored"] = (df.groupby(level=0)["corrected_survival"].min() == 0).astype(int)

In [19]:
df_t = df_flat.copy()
df_t["A"] = 1
df_c = df_flat.copy()
df_c["A"] = 0

### Cross validating...

In [3]:
def mae(df, y_hat):
    a = np.abs((df["total_hours"] - y_hat)[df["uncensored"].astype(bool)]).sum()
    b = np.maximum(np.zeros(df.shape[0]), df["total_hours"] - y_hat).sum()
    return (a + b) / df.shape[0]

In [22]:
predict_treated = []
predict_control = []
for seed in [71, 72, 73, 74, 75, 76, 77]:
    train, val = train_test_split(df_flat, train_size=0.8, random_state=seed)
    val_scores = []
    models = []
    for lam in [0, .1, .2,]:
        cph = CoxPHFitter(penalizer=lam)
        cph.fit(train, duration_col="total_hours", event_col="uncensored")
        models.append(cph)
        y_hat_val = cph.predict_expectation(val)
        val_scores.append(mae(val, y_hat_val))
    best_ix = np.argmin(val_scores)
    best_model = models[best_ix]
    predict_treated.append(best_model.predict_expectation(df_t))
    predict_control.append((best_model.predict_expectation(df_c)))

In [30]:
taus = [8, 12, 16]
for i in range(len(predict_treated)):
    print(i)
    for tau in taus:        
        cutoff = np.full(len(df_flat), tau)
        ey_x1 = np.minimum(cutoff, predict_treated[i])
        ey_x0 = np.minimum(cutoff, predict_control[i])
        print(ey_x1.mean() - ey_x0.mean())

0
0.006990906215971648
0.04817908881326538
0.20784206957636364
1
0.005685950504813242
0.04674804763620877
0.1989215854282982
2
0.005243515192804082
0.04662786868173363
0.19497070653980053
3
0.0054282330270361
0.046855169459851354
0.1981455591943888
4
0.005037709638730625
0.0447034001044706
0.18694627135277742
5
0.0049232193938832935
0.04578574127161161
0.19545287258091193
6
0.003975559105430904
0.045807192170666866
0.18713421894600657


### Propensity Score Weighing

In [10]:
df = pd.read_csv("data/mimic3_df_30.csv", index_col=[0,1])

gender  stay_length  hypertension  coronary_ath  \
subject_id hours_in                                                    
4          0              0    -0.360949             0             0   
           1              0    -0.360949             0             0   
           2              0    -0.360949             0             0   
           3              0    -0.360949             0             0   
           4              0    -0.360949             0             0   

                     atrial_fib  hematocrit  hemoglobin  platelets  \
subject_id hours_in                                                  
4          0                  0    0.076528   -0.111641  -0.187116   
           1                  0    0.076528   -0.111641  -0.187116   
           2                  0    0.076528   -0.111641  -0.187116   
           3                  0    0.076528   -0.111641  -0.187116   
           4                  0    0.076528   -0.111641  -0.187116   

                     mean blood pressure  treated  ...    A  baseline_hazard  \
subject_id hours_in                                ...                         
4          0                    2.523323        1  ...  0.0         0.001000   
           1                    0.082396        1  ...  0.0         0.000779   
           2                    0.082396        1  ...  0.0         0.000607   
           3                   -0.490692        1  ...  0.0         0.000472   
           4                   -0.320892        1  ...  0.0         0.000368   

                       hazard  critical         q  survival_prob  survives  \
subject_id hours_in                                                          
4          0         0.001013         0  0.998987       0.998097         1   
           1         0.000789         0  0.999211       0.999160         1   
           2         0.000614         0  0.999386       0.996084         1   
           3         0.000496         0  0.999504       0.997700         1   
           4         0.000378         0  0.999622       0.996546         1   

                     first_failure censored  corrected_survival  
subject_id hours_in                                              
4          0                   NaN     True                   1  
           1                   NaN     True                   1  
           2                   NaN     True                   1  
           3                   NaN     True                   1  
           4                   NaN     True                   1  

[5 rows x 21 columns]

In [11]:
cols = ["A","hypertension", "coronary_ath", "atrial_fib" ]
df_flat = df[cols].groupby(level=0).head(1)

In [12]:
propensities = []
for seed in [71, 72, 73, 74, 75, 76, 77]:
    lr = LogisticRegressionCV(random_state=seed, max_iter=100)
    lr.fit(df_flat.drop(columns="A"), df_flat["A"])
    lr.score(df_flat.drop(columns="A"), df_flat["A"])
    pi_x = lr.predict_proba(df_flat.drop(columns="A"))[:, 1]
    propensities.append(pi_x)

In [14]:
def ipw(y, t, pi_x):
    p1 = (y * t) / pi_x
    p2 = ((1 - t) * y) / (1 - pi_x)
    return (p1 - p2).mean()

In [15]:
t = df.groupby(level=0)["A"].any().astype(int)
for pi_x in propensities:
    for tau in [8,12,16]:
        surv_restr = df.groupby(level=0)["corrected_survival"].head(tau)
        y = surv_restr.groupby(level=0).sum()
        print(ipw(y, t, pi_x))
    print("***")

0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***
0.5218639886321306
0.9275064793300295
1.3620698921976284
***


### AIPW

In [17]:
def aipw(y, t, pi_x, ey_x1, ey_x0):
    b1 = ((t * y) / pi_x) - (((1 - t) * y) / (1 - pi_x))
    b2 = (t - pi_x) / (pi_x * (1 - pi_x)) * ((1 - pi_x) * ey_x1.numpy() + pi_x * ey_x0.numpy())
    return b1.mean() - b2.mean()

In [18]:
for i in range(len(predict_treated)):
    print(i)
    for tau in [8,12,16]:
        surv_restr = df.groupby(level=0)["corrected_survival"].head(tau)
        y = surv_restr.groupby(level=0).sum()
        cutoff = torch.full(predict_treated[0].shape, tau)
        ey_x1 = torch.minimum(cutoff, predict_treated[i])
        ey_x0 = torch.minimum(cutoff, predict_control[i])
        print(aipw(y, t, pi_x, ey_x1, ey_x0))

0
0.2449780630071885
0.6446655469578944
1.1689455879817172
1
0.20984590972150602
0.5189298083180993
0.9954442420118628
2
0.22329082169686076
0.5758678432394009
1.0657333240910896
3
0.21811878943630775
0.53925737975217
1.011066912574107
4
0.23717831059358457
0.6600351872592145
1.196138547877693
5
0.2226971317555551
0.5723569840637329
1.0625096558344098
